In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install efficientnet_pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=fa92adaa114e9cde4471cd69ff977f958e23c30531a3fda5c50fcec42ffeda55
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [ ]:
import torchvision
from torchvision import transforms
import os
from torch.utils.data import Dataset,DataLoader
import torch

In [ ]:
#바꾸는것 
# 1. PATH 에 숫자
# 2. testset 에 샘플경로

# 모델경로
PATH = '/content/drive/MyDrive/Colab Notebooks/scalp_weights/'+'aram_model6.pt'   # aram_model6.pt 모델파일 이름 바꿔주기 

# Cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load(PATH, map_location=device)

# 전처리-트랜스폼 규칙 선언 # model1_train 코드의 validation set 의 트랜스폼 규칙과 동일하게 함
transforms_test = transforms.Compose([
                                        transforms.Resize([int(600), int(600)], interpolation=transforms.InterpolationMode.BOX),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                      ])
# root 경로 폴더 속 jpg를 전처리, 텐서화 (rood 속에 폴더를 하나 더 만들어서 jpg를 묶어야 함)
testset = torchvision.datasets.ImageFolder(root = '/content/drive/MyDrive/project/sample/model6sample' ,
                    transform = transforms_test)

# DataLoader를 통해 네트워크에 올리기
from torch.utils.data import Dataset,DataLoader 
testloader = DataLoader(testset, batch_size=2, shuffle=False, num_workers=0)

In [ ]:
## 아웃풋, 로스, 프레딕, 아큐러시

# output_list = []
model.eval() # 평가모드로 전환 # 평가모드와 학습모드의 layer 구성이 다르다
correct = 0

# 로스 연산을 위한
import torch.nn.functional as F   # F : 테스트_로스 연산 함수
test_loss = 0
from tqdm import tqdm # 진행률 표시를 위한

if __name__ == '__main__':
    with torch.no_grad(): # 평가할 땐  gradient를 backpropagation 하지 않기 때문에 no grad로 gradient 계산을 막아서 연산 속도를 높인다
            for data, target in tqdm(testloader):                                   
                data, target  = data.to(device), target.to(device) 
                output = model(data)   # model1에 데이터를 넣어서 아웃풋 > [a,b,c,d] 각 0,1,2,3 의 확률값 리턴 가장 큰 것이 pred
                # output_list.append(output);
                test_loss += F.nll_loss(output, target, reduction = 'sum').item()  # test_loss변수에 각 로스를 축적
                pred = output.argmax(dim=1, keepdim=True) # argmax : 리스트에서 최댓값의 인덱스를 뽑아줌 > y값아웃풋인덱
                correct += pred.eq(target.view_as(pred)).sum().item() # accuracy 측정을 위한 변수 # 각 예측이 맞았는지 틀렸는지 correct변수에 축적 맞을 때마다 +1  # # view_as() 함수는 target 텐서를 view_as() 함수 안에 들어가는 인수(pred)의 모양대로 다시 정렬한다. #  view_as() 함수는 target 텐서를 view_as() 함수 안에 들어가는 인수(pred)의 모양대로 다시 정렬한다. #  pred.eq(data) : pred와 data가 일치하는지 검사

test_loss /= len(testloader.dataset)  # 로스축적된 로스를 데이터 수(경로안jpg수)로 나누기

# 아큐러시 출력 ( :.4f 소수점반올림 )
# print('\nTest set Accuracy: {}/{} ({:.4f}%)\n'.format(correct, len(testloader.dataset), 100. * correct / len(testloader.dataset)))  # 축적된 예측값을 데이터 개수로 나누기 *100 > 확률%값

# 로스, 아큐러시 출력
print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(test_loss, correct, len(testloader.dataset), 100. * correct / len(testloader.dataset)))


100%|██████████| 577/577 [06:25<00:00,  1.50it/s]


Test set: Average Loss: -1.4646, Accuracy: 595/1154 (51.5598%)



In [ ]:
# 모델1 
100%|██████████| 238/238 [03:01<00:00,  1.31it/s]
Test set: Average Loss: -1.9017, Accuracy: 289/476 (60.7143%)

# 모델2  


# 모델3  
100%|██████████| 359/359 [03:43<00:00,  1.60it/s]
Test set: Average Loss: -2.2255, Accuracy: 461/718 (64.2061%)

# 모델4 
100%|██████████| 197/197 [03:26<00:00,  1.05s/it]
Test set: Average Loss: -3.4693, Accuracy: 281/394 (71.3198%)

# 모델5  
100%|██████████| 310/310 [03:43<00:00,  1.39it/s]
Test set: Average Loss: -2.0950, Accuracy: 370/620 (59.6774%)

# 모델6  
100%|██████████| 577/577 [06:25<00:00,  1.50it/s]
Test set: Average Loss: -1.4646, Accuracy: 595/1154 (51.5598%)
